# HW11: Meta-Learning in Reinforcement Learning

> - Full Name: **[Your Full Name]**
> - Student ID: **[Your Student ID]**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DeepRLCourse/Homework-11-Questions/blob/main/HW11_Notebook.ipynb)
[![Open In kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://raw.githubusercontent.com/DeepRLCourse/Homework-11-Questions/main/HW11_Notebook.ipynb)

## Overview
This assignment focuses on **Meta-Learning in Reinforcement Learning**, exploring algorithms that enable agents to quickly adapt to new tasks by leveraging experience from related tasks. We'll implement and experiment with:

1. **MAML (Model-Agnostic Meta-Learning)** - Gradient-based meta-learning for RL
2. **RL² (Recurrent Meta-RL)** - Black-box meta-learning using recurrent networks
3. **PEARL (Probabilistic Embeddings)** - Context-based meta-RL with task embeddings
4. **Few-Shot Adaptation** - Rapid learning on new tasks with minimal samples
5. **Task Distributions** - Learning across families of related RL tasks

The goal is to understand how meta-learning enables "learning to learn" and achieves fast adaptation to new tasks.


In [ ]:
# @title Imports and Setup

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import gymnasium as gym
from gymnasium import spaces
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import trange
from collections import defaultdict, deque
import random
from typing import Tuple, List, Dict, Optional
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")


## 1. Meta-Learning Environment Setup

First, let's create a task distribution for meta-learning. We'll use a parameterized environment where tasks differ in reward functions or dynamics, enabling us to test few-shot adaptation.


In [ ]:
class ParameterizedCartPoleEnv(gym.Env):
    """
    Parameterized CartPole environment for meta-learning.
    Tasks differ in pole length, mass, or reward structure.
    """
    
    def __init__(self, task_params=None):
        super().__init__()
        
        # Default task parameters
        self.default_params = {
            'pole_length': 0.5,
            'pole_mass': 0.1,
            'cart_mass': 1.0,
            'gravity': 9.8,
            'reward_scale': 1.0,
            'success_threshold': 195.0
        }
        
        # Set task parameters
        self.params = self.default_params.copy()
        if task_params:
            self.params.update(task_params)
        
        # State space: [cart_pos, cart_vel, pole_angle, pole_vel]
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(4,), dtype=np.float32
        )
        
        # Action space: [push_left, push_right]
        self.action_space = spaces.Discrete(2)
        
        # Environment state
        self.state = None
        self.steps = 0
        self.max_steps = 200
        
    def reset(self, **kwargs):
        """Reset environment with random initial state."""
        # Random initial state
        self.state = np.array([
            np.random.uniform(-0.1, 0.1),  # cart position
            np.random.uniform(-0.1, 0.1),  # cart velocity
            np.random.uniform(-0.1, 0.1),  # pole angle
            np.random.uniform(-0.1, 0.1)   # pole velocity
        ], dtype=np.float32)
        
        self.steps = 0
        return self.state.copy(), {}
    
    def step(self, action):
        """Execute action and return next state."""
        if self.state is None:
            raise RuntimeError("Environment not reset")
        
        # Convert action to force
        force = 10.0 if action == 1 else -10.0
        
        # CartPole dynamics with task parameters
        x, x_dot, theta, theta_dot = self.state
        
        # Physical constants
        g = self.params['gravity']
        m_cart = self.params['cart_mass']
        m_pole = self.params['pole_mass']
        l = self.params['pole_length']
        
        # Total mass
        total_mass = m_cart + m_pole
        
        # Dynamics
        temp = (force + m_pole * l * theta_dot**2 * np.sin(theta)) / total_mass
        theta_acc = (g * np.sin(theta) - np.cos(theta) * temp) / \
                   (l * (4/3 - m_pole * np.cos(theta)**2 / total_mass))
        x_acc = temp - m_pole * l * theta_acc * np.cos(theta) / total_mass
        
        # Update state
        x = x + x_dot * 0.02
        x_dot = x_dot + x_acc * 0.02
        theta = theta + theta_dot * 0.02
        theta_dot = theta_dot + theta_acc * 0.02
        
        self.state = np.array([x, x_dot, theta, theta_dot], dtype=np.float32)
        self.steps += 1
        
        # Compute reward
        reward = self._compute_reward()
        
        # Check termination
        terminated = self._is_terminated()
        truncated = self.steps >= self.max_steps
        
        return self.state.copy(), reward, terminated, truncated, {}
    
    def _compute_reward(self):
        """Compute reward based on task parameters."""
        x, x_dot, theta, theta_dot = self.state
        
        # Base reward: stay upright and centered
        reward = 1.0
        
        # Penalty for being far from center
        reward -= abs(x) * 0.1
        
        # Penalty for large angle
        reward -= abs(theta) * 0.1
        
        # Scale reward
        reward *= self.params['reward_scale']
        
        return reward
    
    def _is_terminated(self):
        """Check if episode should terminate."""
        x, x_dot, theta, theta_dot = self.state
        
        # Terminate if cart goes too far
        if abs(x) > 2.4:
            return True
        
        # Terminate if pole falls too far
        if abs(theta) > 0.2095:  # ~12 degrees
            return True
        
        return False
    
    def set_task_params(self, task_params):
        """Set new task parameters."""
        self.params.update(task_params)


class TaskDistribution:
    """
    Distribution over parameterized tasks for meta-learning.
    """
    
    def __init__(self, task_type='pole_length'):
        self.task_type = task_type
        
        if task_type == 'pole_length':
            self.param_ranges = {
                'pole_length': (0.3, 0.8),
                'reward_scale': (0.8, 1.2)
            }
        elif task_type == 'mass':
            self.param_ranges = {
                'pole_mass': (0.05, 0.2),
                'cart_mass': (0.8, 1.5),
                'reward_scale': (0.8, 1.2)
            }
        elif task_type == 'gravity':
            self.param_ranges = {
                'gravity': (8.0, 12.0),
                'reward_scale': (0.8, 1.2)
            }
        elif task_type == 'mixed':
            self.param_ranges = {
                'pole_length': (0.3, 0.8),
                'pole_mass': (0.05, 0.2),
                'gravity': (8.0, 12.0),
                'reward_scale': (0.8, 1.2)
            }
    
    def sample_task(self):
        """Sample a random task from the distribution."""
        task_params = {}
        for param, (low, high) in self.param_ranges.items():
            task_params[param] = np.random.uniform(low, high)
        
        return ParameterizedCartPoleEnv(task_params)
    
    def sample_tasks(self, n_tasks):
        """Sample multiple tasks."""
        return [self.sample_task() for _ in range(n_tasks)]


# Test the environment
print("Testing Parameterized CartPole Environment...")

# Test different task types
task_types = ['pole_length', 'mass', 'gravity', 'mixed']
for task_type in task_types:
    print(f"\nTesting {task_type} tasks:")
    task_dist = TaskDistribution(task_type)
    
    # Sample a few tasks
    for i in range(3):
        task = task_dist.sample_task()
        obs, _ = task.reset()
        
        # Run a few steps
        total_reward = 0
        for step in range(10):
            action = task.action_space.sample()
            obs, reward, terminated, truncated, _ = task.step(action)
            total_reward += reward
            
            if terminated or truncated:
                break
        
        print(f"  Task {i+1}: Reward = {total_reward:.2f}, Params = {task.params}")

print("\nEnvironment test completed!")


## 2. MAML (Model-Agnostic Meta-Learning) Implementation

MAML finds initialization parameters that are good for fine-tuning on new tasks. It uses a two-level optimization process:
- **Inner Loop**: Adapt to specific task using gradient descent
- **Outer Loop**: Optimize initialization for fast adaptation across tasks
